In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler





mpl.rcParams['figure.figsize']= (10,8)
mpl.rcParams['axes.grid']= False



In [3]:
df = pd.read_csv(r"C:\Users\temp\ML\Projects\TimeSeries\LSTM_multivariate\energydata_complete.csv")

In [32]:
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [26]:
df['date']= pd.to_datetime(df['date'], infer_datetime_format=True)

In [27]:
df['date']

0       2016-01-11 17:00:00
1       2016-01-11 17:10:00
2       2016-01-11 17:20:00
3       2016-01-11 17:30:00
4       2016-01-11 17:40:00
                ...        
19730   2016-05-27 17:20:00
19731   2016-05-27 17:30:00
19732   2016-05-27 17:40:00
19733   2016-05-27 17:50:00
19734   2016-05-27 18:00:00
Name: date, Length: 19735, dtype: datetime64[ns]

In [33]:
df.set_index(['date'])[['Appliances', 'lights', 'T_out', 'RH_1', 'Visibility']].plot(subplots=True)

array([<AxesSubplot:xlabel='date'>, <AxesSubplot:xlabel='date'>,
       <AxesSubplot:xlabel='date'>, <AxesSubplot:xlabel='date'>,
       <AxesSubplot:xlabel='date'>], dtype=object)

In [63]:
df_input =df[['Appliances', 'T_out', 'RH_1', 'Visibility']]

In [64]:
df_input.describe()

Appliances         T_out          RH_1    Visibility
count  19735.000000  19735.000000  19735.000000  19735.000000
mean      97.694958      7.411665     40.259739     38.330834
std      102.524891      5.317409      3.979299     11.794719
min       10.000000     -5.000000     27.023333      1.000000
25%       50.000000      3.666667     37.333333     29.000000
50%       60.000000      6.916667     39.656667     40.000000
75%      100.000000     10.408333     43.066667     40.000000
max     1080.000000     26.100000     63.360000     66.000000

In [65]:
df_input[df.Appliances>500]

Appliances      T_out       RH_1  Visibility
11            580   5.983333  46.396667   40.000000
314           520   2.900000  41.530000   36.333333
392           550   2.033333  43.433333   40.000000
393           690   2.100000  43.766667   40.000000
394           620   2.166667  43.693333   40.000000
...           ...        ...        ...         ...
19586         600  20.833333  44.066667   40.000000
19587         520  20.900000  43.366667   40.000000
19684         580  15.900000  47.226667   61.000000
19685         660  16.100000  47.693333   61.000000
19695         620  19.100000  48.326667   64.000000

[288 rows x 4 columns]

In [66]:
scaler = MinMaxScaler()

df_scaled =  scaler.fit_transform(df_input)

In [67]:
df_scaled

array([[0.04672897, 0.37299035, 0.56618659, 0.95384615],
       [0.04672897, 0.36923901, 0.54132648, 0.89487179],
       [0.03738318, 0.36548767, 0.53050179, 0.83589744],
       ...,
       [0.24299065, 0.88317256, 0.53866618, 0.37435897],
       [0.38317757, 0.87888532, 0.54949087, 0.38717949],
       [0.39252336, 0.87459807, 0.53875791, 0.4       ]])

In [68]:
features = df_scaled

target = df_scaled[:,0]

In [69]:
TimeseriesGenerator(features, target, length=2, sampling_rate=1, batch_size=1)[0]

(array([[[0.04672897, 0.37299035, 0.56618659, 0.95384615],
         [0.04672897, 0.36923901, 0.54132648, 0.89487179]]]),
 array([0.03738318]))

In [70]:
## Split the data

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=123, shuffle=False)

In [72]:
x_train.shape

(15788, 4)

In [73]:
x_test.shape

(3947, 4)

In [74]:
from re import L


win_length = 720 ## 5 days, 10 min 
batch_size =32
num_features = 4

def ts_generator(features, target,batch_size, win_length,sampling_rate=1):
    return (TimeseriesGenerator(features, target, length=win_length, sampling_rate=1, batch_size=batch_size))

## Using TS generator for developing train and test time series models
train_generator= ts_generator(x_train, y_train,batch_size, win_length)
test_generator= ts_generator(x_test, y_test,batch_size, win_length)


In [75]:
## LSTM Model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(64, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 720, 128)          68096     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 720, 128)          0         
                                                                 
 lstm_7 (LSTM)               (None, 720, 128)          131584    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 720, 128)          0         
                                                                 
 dropout_4 (Dropout)         (None, 720, 128)          0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_5 (Dropout)         (None, 64)               

In [76]:
from pickletools import optimize
from random import shuffle


early_Stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=2, 
                                                  mode='min')

model.compile(loss=tf.losses.MeanSquaredError(), 
                optimizer= tf.optimizers.Adam(),
                metrics = [tf.metrics.MeanAbsoluteError()])

history =  model.fit_generator(train_generator, epochs=50, 
                            validation_data=test_generator, 
                            shuffle=False, 
                            callbacks=[early_Stopping])

Epoch 1/50


C:\Users\temp\AppData\Local\Temp/ipykernel_7768/1181526285.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history =  model.fit_generator(train_generator, epochs=50,


 21/471 [>.............................] - ETA: 7:03 - loss: 0.0193 - mean_absolute_error: 0.0849

KeyboardInterrupt: 

In [81]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2383950127713571153
xla_global_id: -1
]


In [82]:
tf.config.list_physical_devices('GPU')

[]